In [ ]:
!pip install -r requirements.txt

In [1]:
MODEL_CHECKPOINTS_PATH = 'models/checkpoints'
MODEL_ARCHITECTURE_PATH = 'models/model_architecture'
MODEL_WEIGHTS_PATH = 'models/model_weights'
MODEL_HISTORY_PATH = 'models/model_history'
MODEL_SERVING_PATH = 'models/tf_saved_models/1'

#IMAGE_SIZE = 256
IMAGE_SIZE = 300
IMAGE_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

LABELS = ['No', 'Yes']

EPOCHS = 20
BATCH_SIZE = 15
LEARNING_RATE = 1e-4

### Load Data

In [2]:
train_dir = 'data/2021_STUDY/HE/train'
val_dir = 'data/2021_STUDY/HE/eval'

---

In [3]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, BatchNormalization, Dense, Dropout
from tensorflow.keras.optimizers import Adam

def create_model(input_shape, learning_rate=1e-4):
    pretrained_model = ResNet50(
        weights='imagenet',  # Load weights pre-trained on ImageNet.
        input_shape=input_shape,
        include_top=False
    )
    # freeze the the pre-trained model
    #pretrained_model.trainable = False
    for layer in pretrained_model.layers:
        if isinstance(layer, BatchNormalization):
            layer.trainable = True
        else:
            layer.trainable = False
    model = Sequential([
        pretrained_model,
        GlobalAveragePooling2D(),
        Dense(128, activation='relu'),
        Dropout(rate=0.3),
        Dense(16, activation='relu'),
        Dropout(rate=0.3),
        Dense(1, activation='sigmoid')
    ])
    model.compile(
        optimizer=Adam(lr=learning_rate),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

model = create_model(
    IMAGE_SHAPE,
    learning_rate=LEARNING_RATE
)
model.summary()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 10, 10, 2048)      23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               262272    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                2064      
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
____________________________

In [5]:
import os

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input

def train(model, train_dir, val_dir, input_shape,
          batch_size=15,
          epochs=50):
    """
    Train a Keras model usin batches.
    
    Arguments:
      model -- Keras model to train.
      train_gen -- Python generator to retrieve the training samples by batches.
      eval_gen -- Python generator to retrieve the validation samples by batches.
      input_shape -- tuple indicating the image shape (high, width, num_channels).
      batch_size -- number of images to retrieve in each batch.
      epochs -- numbers of time that the model have to train the whole dataset.
    
    Returns:
      model -- Keras model trained.
    """
    if not os.path.isdir(MODEL_CHECKPOINTS_PATH):
        os.mkdir(MODEL_CHECKPOINTS_PATH)
    filepath = 'models/checkpoints/weights-{epoch:02d}-{val_acc:.2f}.hdf5'
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='val_acc',
        verbose=0,
        save_best_only=True,
        save_weights_only=False,
        mode='max'
    )
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        preprocessing_function=preprocess_input
    )
    train_gen = train_datagen.flow_from_directory(
        train_dir,
        target_size=input_shape,
        batch_size=batch_size,
        class_mode='binary',
        shuffle=True,
        seed=16
    )
    val_datagen = ImageDataGenerator(
        rescale=1./255,
        preprocessing_function=preprocess_input
    )
    val_gen = val_datagen.flow_from_directory(
        val_dir,
        target_size=input_shape,
        batch_size=batch_size,
        class_mode='binary',
        shuffle=True,
        seed=16
    )
    train_steps = train_gen.n // train_gen.batch_size
    validation_steps = val_gen.n // val_gen.batch_size
    return model.fit_generator(
        train_gen, 
        epochs=epochs,
        steps_per_epoch=train_steps,
        validation_data=val_gen,
        validation_steps=validation_steps,
        callbacks=[checkpoint]
    )

history = train(
    model,
    train_dir,
    val_dir,
    (IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    epochs=EPOCHS
)

Found 20210 images belonging to 2 classes.
Found 4030 images belonging to 2 classes.
Epoch 1/20
1347/1347 [==============================] - 13619s 10s/step - loss: 0.3880 - acc: 0.8252 - val_loss: 0.3114 - val_acc: 0.8692
Epoch 2/20
1347/1347 [==============================] - 13128s 10s/step - loss: 0.2259 - acc: 0.9109 - val_loss: 0.3134 - val_acc: 0.8624
Epoch 3/20
1347/1347 [==============================] - 13083s 10s/step - loss: 0.1535 - acc: 0.9432 - val_loss: 0.3095 - val_acc: 0.8726
Epoch 4/20
1347/1347 [==============================] - 13158s 10s/step - loss: 0.1128 - acc: 0.9608 - val_loss: 0.3438 - val_acc: 0.8789
Epoch 5/20
1347/1347 [==============================] - 19788s 15s/step - loss: 0.0802 - acc: 0.9716 - val_loss: 0.3801 - val_acc: 0.8697
Epoch 6/20
1347/1347 [==============================] - 15543s 12s/step - loss: 0.0623 - acc: 0.9790 - val_loss: 0.3628 - val_acc: 0.8945
Epoch 7/20
1347/1347 [==============================] - 11899s 9s/step - loss: 0.0529 -

---

In [6]:
import pickle

def save_model(model):
    """
    Saves a Keras model creating the following file structure:
      model_architecture -- neural network architecture saved as JSON and Yaml files.
      model_history -- model training history saved as Pickle binary object.
      model_weights -- model weights saved in HDF5 format.
    
    Arguments:
      model -- Keras model to train.
    """
    # Saves the whole model
    model_path = 'models/resnet-{}.h5'.format(IMAGE_SIZE)
    model.save(model_path)
    
    if not os.path.isdir(MODEL_ARCHITECTURE_PATH):
        os.mkdir(MODEL_ARCHITECTURE_PATH)

    # Saves the model architecture as JSON file
    json_path = '{}/resnet-{}-arc.json'.format(MODEL_ARCHITECTURE_PATH, IMAGE_SIZE)
    with open(json_path, "w") as json_file:
        json_file.write(model.to_json())

    # Saves the model architecture as YAML file
    yaml_path = '{}/resnet-{}-arc.yml'.format(MODEL_ARCHITECTURE_PATH, IMAGE_SIZE)
    with open(yaml_path, "w") as yaml_file:
        yaml_file.write(model.to_yaml())
        
    print("Model architecture saved as:\n\t- {}\n\t- {}".format(json_path, yaml_path))
        
    if not os.path.isdir(MODEL_WEIGHTS_PATH):
        os.mkdir(MODEL_WEIGHTS_PATH)

    # Saves the model weights
    weight_path = '{}/resnet-{}-wt.h5'.format(MODEL_WEIGHTS_PATH, IMAGE_SIZE)
    model.save_weights(
        filepath=weight_path,
        overwrite=True
    )
    
    print("Model weights saved as {}.".format(weight_path))
    
    if not os.path.isdir(MODEL_HISTORY_PATH):
        os.mkdir(MODEL_HISTORY_PATH)

    # Saves the history model
    history_path = '{}/resnet-{}-history'.format(MODEL_HISTORY_PATH, IMAGE_SIZE)
    with open(history_path, 'wb') as f:
        pickle.dump(model.history.history, f)
        
    print("Model history saved as {}.".format(history_path))

In [7]:
from tensorflow.keras.models import model_from_json, model_from_yaml

def load_model(arc_path, weights_path=None):
    """
    Loads a model which was saved with save_model function.
    
    Arguments:
      arc_path -- path to the model architecture file (JSON or YAML).
      weights_path -- path to the model weights file (HDF5).
      
    Returns:
      model -- loaded Keras model.
    """
    file = open(arc_path, 'r')
    content = file.read()
    model = None
    
    if '.json' in arc_path:
        model = model_from_json(content)
    elif '.yml' in arc_path:
        model = model_from_yaml(content)
    
    if weights_path is not None:
        model.load_weights(weights_path)
    
    return model

In [8]:
save_model(model)

Model architecture saved as:
	- models/model_architecture/resnet-300-arc.json
	- models/model_architecture/resnet-300-arc.yml
Model weights saved as models/model_weights/resnet-300-wt.h5.
Model history saved as models/model_history/resnet-300-history.


In [9]:
json_path = '{}/resnet-{}-arc.json'.format(MODEL_ARCHITECTURE_PATH, IMAGE_SIZE)
saved_model = load_model(json_path)

print("Keras Model loaded from a JSON file.")

Keras Model loaded from a JSON file.


In [12]:
!gsutil -m mv gs://ex-vivo-confocal/models/HE gs://ex-vivo-confocal/models/HE_back
!gsutil -m cp -r models/HE gs://ex-vivo-confocal/models/HE

Copying file://models/HE/resnet-300.h5 [Content-Type=application/octet-stream]...
Copying file://models/HE/model_architecture/resnet-300-arc.yml [Content-Type=application/octet-stream]...
Copying file://models/HE/model_architecture/resnet-300-arc.json [Content-Type=application/json]...
Copying file://models/HE/model_history/resnet-300-history [Content-Type=application/octet-stream]...
Copying file://models/HE/model_weights/resnet-300-wt.h5 [Content-Type=application/octet-stream]...
\ [5/5 files][185.4 MiB/185.4 MiB] 100% Done  15.6 MiB/s ETA 00:00:00           
Operation completed over 5 objects/185.4 MiB.                                    


---